<a href="https://colab.research.google.com/github/saktiworkstation/road-to-ai-developer/blob/main/Pyspark_Join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=39b0924b8c9db6533989d9f9e15041642cec54f4c6bca4651e9c57bc7ceca518
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


# Materi
# Sakti Kusumaa Aji | 22.11.4677

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Buat Spark session
spark = SparkSession.builder.master("local").appName("Dataset pesawat").getOrCreate()

In [10]:
# Load dataset dari file CSV menggunakan spark.read.csv dan pastikan header diakui
airport = spark.read.csv("/content/airports.csv", header=True, inferSchema=True)
flights = spark.read.csv("/content/flights_small.csv", header=True, inferSchema=True)
planes = spark.read.csv("/content/planes.csv", header=True, inferSchema=True)

In [11]:
# Tampilkan 5 baris pertama dari tiap dataset dengan lebih rapi
print("Airport Dataset:")
airport.show(5, truncate=False)

print("Flights Dataset:")
flights.show(5, truncate=False)

print("Planes Dataset:")
planes.show(5, truncate=False)

Airport Dataset:
+---+-----------------------------+----------+-----------+----+---+---+
|faa|name                         |lat       |lon        |alt |tz |dst|
+---+-----------------------------+----------+-----------+----+---+---+
|04G|Lansdowne Airport            |41.1304722|-80.6195833|1044|-5 |A  |
|06A|Moton Field Municipal Airport|32.4605722|-85.6800278|264 |-5 |A  |
|06C|Schaumburg Regional          |41.9893408|-88.1012428|801 |-6 |A  |
|06N|Randall Airport              |41.431912 |-74.3915611|523 |-5 |A  |
|09J|Jekyll Island Airport        |31.0744722|-81.4277778|11  |-4 |A  |
+---+-----------------------------+----------+-----------+----+---+---+
only showing top 5 rows

Flights Dataset:
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------

In [14]:
# Inner Join antara airport dan flights berdasarkan kolom origin dari flights dan faa dari airport
inner_join_df = flights.join(airport, flights.origin == airport.faa, how='inner')

print("Inner Join:")
inner_join_df.show(5, truncate=False)

Inner Join:
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|faa|name               |lat      |lon        |alt|tz |dst|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |SEA|Seattle Tacoma Intl|47.449   |-122.309306|433|-8 |A  |
|2014|1    |22 |1040    |5        |1505    |5        |AS     |N559AS |851   |SEA   |HNL |360     |2677    |10  |40    |SEA|Seattle Tacoma Intl|47.449   |-122.309306|433|-8 |A  |
|2014|3    |9  |1443    |-2       |1652    |2        |VX     |N847VA |755   |SEA   |SFO |111     |

In [16]:
# Filtering: Menampilkan flight yang memiliki delay lebih dari 0
inner_filter_df = inner_join_df.filter(col("dep_delay") > 0)
print("Inner Join with Filter (dep_delay > 0):")
inner_filter_df.show(5, truncate=False)

Inner Join with Filter (dep_delay > 0):
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|faa|name               |lat      |lon        |alt|tz |dst|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|2014|1    |22 |1040    |5        |1505    |5        |AS     |N559AS |851   |SEA   |HNL |360     |2677    |10  |40    |SEA|Seattle Tacoma Intl|47.449   |-122.309306|433|-8 |A  |
|2014|4    |9  |1705    |45       |1839    |34       |WN     |N360SW |344   |PDX   |SJC |83      |569     |17  |5     |PDX|Portland Intl      |45.588722|-122.5975  |30 |-8 |A  |
|2014|1    |15 |1037    |7        |1352    |2        |WN     |N646SW |

In [17]:
# Left Join antara airport dan flights
left_join_df = flights.join(airport, flights.origin == airport.faa, how='left')
print("Left Join:")
left_join_df.show(5, truncate=False)

Left Join:
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|faa|name               |lat      |lon        |alt|tz |dst|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |SEA|Seattle Tacoma Intl|47.449   |-122.309306|433|-8 |A  |
|2014|1    |22 |1040    |5        |1505    |5        |AS     |N559AS |851   |SEA   |HNL |360     |2677    |10  |40    |SEA|Seattle Tacoma Intl|47.449   |-122.309306|433|-8 |A  |
|2014|3    |9  |1443    |-2       |1652    |2        |VX     |N847VA |755   |SEA   |SFO |111     |6

In [18]:
# Filtering: Menampilkan flight yang memiliki dep_delay di antara -5 dan 10, air_time lebih dari 100, dan ketinggian alt di bawah 1000
left_filter_df = left_join_df.filter((col("dep_delay").between(-5, 10)) & (col("air_time") > 100) & (col("alt") < 1000))
print("Left Join with Complex Filter (dep_delay between -5 and 10, air_time > 100, alt < 1000):")
left_filter_df.show(5, truncate=False)

Left Join with Complex Filter (dep_delay between -5 and 10, air_time > 100, alt < 1000):
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|faa|name               |lat      |lon        |alt|tz |dst|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|2014|1    |22 |1040    |5        |1505    |5        |AS     |N559AS |851   |SEA   |HNL |360     |2677    |10  |40    |SEA|Seattle Tacoma Intl|47.449   |-122.309306|433|-8 |A  |
|2014|3    |9  |1443    |-2       |1652    |2        |VX     |N847VA |755   |SEA   |SFO |111     |679     |14  |43    |SEA|Seattle Tacoma Intl|47.449   |-122.309306|433|-8 |A  |
|2014|3    |9  |754  

In [19]:
# Right Join antara airport dan flights
right_join_df = flights.join(airport, flights.origin == airport.faa, how='right')
print("Right Join:")
right_join_df.show(5, truncate=False)

Right Join:
+----+-----+----+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-----------------------------+----------+-----------+----+---+---+
|year|month|day |dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|faa|name                         |lat       |lon        |alt |tz |dst|
+----+-----+----+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-----------------------------+----------+-----------+----+---+---+
|NULL|NULL |NULL|NULL    |NULL     |NULL    |NULL     |NULL   |NULL   |NULL  |NULL  |NULL|NULL    |NULL    |NULL|NULL  |04G|Lansdowne Airport            |41.1304722|-80.6195833|1044|-5 |A  |
|NULL|NULL |NULL|NULL    |NULL     |NULL    |NULL     |NULL   |NULL   |NULL  |NULL  |NULL|NULL    |NULL    |NULL|NULL  |06A|Moton Field Municipal Airport|32.4605722|-85.6800278|264 |-5 |A  |
|NULL|NULL |NULL|NULL    |NULL   

In [22]:
# Filtering: Menampilkan flight yang memiliki alt lebih dari 500 (ketinggian)
right_filter_df = right_join_df.filter(col("alt") > 500)
print("Right Join with Filter (alt > 500):")
right_filter_df.show(5, truncate=False)

Right Join with Filter (alt > 500):
+----+-----+----+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+------------------------------+----------+-----------+----+---+---+
|year|month|day |dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|faa|name                          |lat       |lon        |alt |tz |dst|
+----+-----+----+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+------------------------------+----------+-----------+----+---+---+
|NULL|NULL |NULL|NULL    |NULL     |NULL    |NULL     |NULL   |NULL   |NULL  |NULL  |NULL|NULL    |NULL    |NULL|NULL  |04G|Lansdowne Airport             |41.1304722|-80.6195833|1044|-5 |A  |
|NULL|NULL |NULL|NULL    |NULL     |NULL    |NULL     |NULL   |NULL   |NULL  |NULL  |NULL|NULL    |NULL    |NULL|NULL  |06C|Schaumburg Regional           |41.9893408|-88.1012428|801 |-6 |A  |
|NUL

In [23]:
# Full Outer Join antara airport dan flights
outer_join_df = flights.join(airport, flights.origin == airport.faa, how='outer')
print("Full Outer Join:")
outer_join_df.show(5, truncate=False)

Full Outer Join:
+----+-----+----+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-----------------------------+----------+-----------+----+---+---+
|year|month|day |dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|faa|name                         |lat       |lon        |alt |tz |dst|
+----+-----+----+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-----------------------------+----------+-----------+----+---+---+
|NULL|NULL |NULL|NULL    |NULL     |NULL    |NULL     |NULL   |NULL   |NULL  |NULL  |NULL|NULL    |NULL    |NULL|NULL  |04G|Lansdowne Airport            |41.1304722|-80.6195833|1044|-5 |A  |
|NULL|NULL |NULL|NULL    |NULL     |NULL    |NULL     |NULL   |NULL   |NULL  |NULL  |NULL|NULL    |NULL    |NULL|NULL  |06A|Moton Field Municipal Airport|32.4605722|-85.6800278|264 |-5 |A  |
|NULL|NULL |NULL|NULL    |NU

In [24]:
# Filtering: Menampilkan flight yang memiliki arrival delay (arr_delay) lebih dari 10
outer_filter_df = outer_join_df.filter(col("arr_delay") > 10)
print("Full Outer Join with Filter (arr_delay > 10):")
outer_filter_df.show(5, truncate=False)

Full Outer Join with Filter (arr_delay > 10):
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|faa|name               |lat      |lon        |alt|tz |dst|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+---+-------------------+---------+-----------+---+---+---+
|2014|4    |9  |1705    |45       |1839    |34       |WN     |N360SW |344   |PDX   |SJC |83      |569     |17  |5     |PDX|Portland Intl      |45.588722|-122.5975  |30 |-8 |A  |
|2014|7    |2  |847     |42       |1041    |51       |WN     |N422WN |1520  |PDX   |OAK |90      |543     |8   |47    |PDX|Portland Intl      |45.588722|-122.5975  |30 |-8 |A  |
|2014|10   |30 |811     |21       |1038    |29       |AS     |N4

In [25]:
# Cross Join antara airport dan planes
cross_join_df = airport.crossJoin(planes)
print("Cross Join:")
cross_join_df.show(5, truncate=False)

Cross Join:
+---+-----------------------------+----------+-----------+----+---+---+-------+----+-----------------------+----------------+--------+-------+-----+-----+---------+
|faa|name                         |lat       |lon        |alt |tz |dst|tailnum|year|type                   |manufacturer    |model   |engines|seats|speed|engine   |
+---+-----------------------------+----------+-----------+----+---+---+-------+----+-----------------------+----------------+--------+-------+-----+-----+---------+
|04G|Lansdowne Airport            |41.1304722|-80.6195833|1044|-5 |A  |N102UW |1998|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|06A|Moton Field Municipal Airport|32.4605722|-85.6800278|264 |-5 |A  |N102UW |1998|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan|
|06C|Schaumburg Regional          |41.9893408|-88.1012428|801 |-6 |A  |N102UW |1998|Fixed wing multi engine|AIRBUS INDUSTRIE|A320-214|2      |182  |NA   |Turbo-fan

In [26]:
# Filtering: Menampilkan data pesawat yang memiliki lebih dari 2 mesin (engines)
cross_filter_df = cross_join_df.filter(col("engines") > 2)
print("Cross Join with Filter (engines > 2):")
cross_filter_df.show(5, truncate=False)

Cross Join with Filter (engines > 2):
+---+-----------------------------+----------+-----------+----+---+---+-------+----+-----------------------+------------+-------+-------+-----+-----+---------+
|faa|name                         |lat       |lon        |alt |tz |dst|tailnum|year|type                   |manufacturer|model  |engines|seats|speed|engine   |
+---+-----------------------------+----------+-----------+----+---+---+-------+----+-----------------------+------------+-------+-------+-----+-----+---------+
|04G|Lansdowne Airport            |41.1304722|-80.6195833|1044|-5 |A  |N675NW |2002|Fixed wing multi engine|BOEING      |747-451|4      |450  |NA   |Turbo-jet|
|06A|Moton Field Municipal Airport|32.4605722|-85.6800278|264 |-5 |A  |N675NW |2002|Fixed wing multi engine|BOEING      |747-451|4      |450  |NA   |Turbo-jet|
|06C|Schaumburg Regional          |41.9893408|-88.1012428|801 |-6 |A  |N675NW |2002|Fixed wing multi engine|BOEING      |747-451|4      |450  |NA   |Turbo-jet|
|0

# Studi Kasus

In [39]:
# Inner Join antara flights dan planes berdasarkan kolom 'year'
flights_with_planes = flights.join(planes, flights.year == planes.year, how='inner')
flights_with_planes.show(5, truncate=False)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+----+-----------------------+------------+--------+-------+-----+-----+---------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|tailnum|year|type                   |manufacturer|model   |engines|seats|speed|engine   |
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+----+-----------------------+------------+--------+-------+-----+-----+---------+
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |N914UY |2014|Fixed wing multi engine|AIRBUS      |A321-231|2      |379  |NA   |Turbo-fan|
|2014|12   |8  |658     |-7       |935     |-5       |VX     |N846VA |1780  |SEA   |LAX |132     |954     |6   |58    |N913US |2014|Fixed wing multi engine|AIRBUS  

In [32]:
# dep_time, arr_time, origin, dan dest dengan filter dep_time < 1000 dan origin = 'PDX'
filtered_flights = flights_with_planes.select("dep_time", "arr_time", "origin", "dest").filter((col("dep_time") < 1000) & (col("origin") == 'PDX'))
filtered_flights.show()

+--------+--------+------+----+
|dep_time|arr_time|origin|dest|
+--------+--------+------+----+
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
|     847|    1041|   PDX| OAK|
+--------+--------+------+----+
only showing top 20 rows



In [36]:
# Inner Join antara flights dan planes berdasarkan kolom 'tailnum'
flights_with_planes = flights.join(planes, flights.tailnum == planes.tailnum, how='inner')

# Menghitung rata-rata waktu penerbangan (air_time) untuk setiap model pesawat (model)
avg_airtime_by_model = flights_with_planes.groupBy("model").agg({"air_time": "avg"}).withColumnRenamed("avg(air_time)", "average_air_time")

# Tampilkan rata-rata air_time per model dengan hasil yang benar
avg_airtime_by_model.filter(col("average_air_time").isNotNull()).show(truncate=False)


+-----------+------------------+
|model      |average_air_time  |
+-----------+------------------+
|737-76Q    |116.2             |
|737-3A4    |135.0             |
|EMB-120ER  |33.92946058091286 |
|757-2S7    |129.0             |
|A321-231   |200.9777777777778 |
|737-705    |209.5             |
|737-924    |196.33333333333334|
|421C       |199.33333333333334|
|A320-211   |127.12280701754386|
|A320-232   |183.10310965630114|
|A319-112   |128.06024096385542|
|EMB-120    |34.78260869565217 |
|737-824    |171.17222222222222|
|A330-223   |230.5             |
|CL-600-2D24|87.31192660550458 |
|757-223    |250.375           |
|757-232    |159.64864864864865|
|A320-214   |135.2829457364341 |
|737-301    |84.0              |
|737-932ER  |204.63924050632912|
+-----------+------------------+
only showing top 20 rows



In [30]:
# Menghitung rata-rata keterlambatan kedatangan (arr_delay) untuk setiap maskapai (carrier) dan urutkan descending
avg_arr_delay_by_carrier = flights.groupBy("carrier").agg({"arr_delay": "avg"}).orderBy(col("avg(arr_delay)").desc())
avg_arr_delay_by_carrier.show()

+-------+-------------------+
|carrier|     avg(arr_delay)|
+-------+-------------------+
|     AA|   6.51890756302521|
|     WN|   6.04985549132948|
|     F9|  5.392265193370166|
|     UA|  3.217516843118383|
|     B6| 2.6291079812206575|
|     HA| 2.2816901408450705|
|     OO| 1.9260533104041273|
|     VX|  1.564516129032258|
|     DL| 1.3955431754874652|
|     AS| 0.6234420578095996|
|     US|-1.4835164835164836|
+-------+-------------------+

